In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF
from sklearn.datasets import fetch_20newsgroups
from sklearn.metrics.pairwise import cosine_similarity as cosine
import itertools
import json
import numpy as np

In [3]:

n_topics = 10
n_top_words = 20
dataset = fetch_20newsgroups(shuffle=True, random_state=1,
                             remove=('headers', 'footers', 'quotes'))
data_samples = dataset.data


In [4]:
print("Extracting tf-idf features for NMF...")
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, #max_features=n_features,
                                   stop_words='english')

tfidf = tfidf_vectorizer.fit_transform(data_samples)


nmf = NMF(n_components=n_topics).fit(tfidf)

Extracting tf-idf features for NMF...


In [62]:
local_data = []
philes =  glob.glob("/Users/ziv/GDrive/school/thesis/nmf-imp/data/*.txt")
for phile in philes:
    print phile
    with open(phile, 'r') as myfile:
        data=myfile.read().replace('\n', '')
        local_data.append(data)

/Users/ziv/GDrive/school/thesis/nmf-imp/data/cinc1.txt
/Users/ziv/GDrive/school/thesis/nmf-imp/data/cinc10.txt
/Users/ziv/GDrive/school/thesis/nmf-imp/data/cinc2.txt
/Users/ziv/GDrive/school/thesis/nmf-imp/data/cinc3.txt
/Users/ziv/GDrive/school/thesis/nmf-imp/data/cinc4.txt
/Users/ziv/GDrive/school/thesis/nmf-imp/data/cinc5.txt
/Users/ziv/GDrive/school/thesis/nmf-imp/data/cinc6.txt
/Users/ziv/GDrive/school/thesis/nmf-imp/data/cinc7.txt
/Users/ziv/GDrive/school/thesis/nmf-imp/data/cinc8.txt
/Users/ziv/GDrive/school/thesis/nmf-imp/data/cinc9.txt
/Users/ziv/GDrive/school/thesis/nmf-imp/data/dotm1.txt
/Users/ziv/GDrive/school/thesis/nmf-imp/data/dotm10.txt
/Users/ziv/GDrive/school/thesis/nmf-imp/data/dotm11.txt
/Users/ziv/GDrive/school/thesis/nmf-imp/data/dotm12.txt
/Users/ziv/GDrive/school/thesis/nmf-imp/data/dotm13.txt
/Users/ziv/GDrive/school/thesis/nmf-imp/data/dotm14.txt
/Users/ziv/GDrive/school/thesis/nmf-imp/data/dotm15.txt
/Users/ziv/GDrive/school/thesis/nmf-imp/data/dotm2.txt
/Us

In [5]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic #%d:" % topic_idx)
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))
    print()
print("\nTopics in NMF model:")
tfidf_feature_names = tfidf_vectorizer.get_feature_names()
print_top_words(nmf, tfidf_feature_names, n_top_words)


Topics in NMF model:
Topic #0:
game team games year players season play hockey win league teams baseball nhl player good detroit toronto runs best pitching
Topic #1:
god jesus bible believe faith christ christian christians church does life say heaven truth sin hell lord belief man christianity
Topic #2:
key chip encryption clipper keys government escrow algorithm use law secure security public nsa bit encrypted des phone privacy enforcement
Topic #3:
geb dsl n3jxp chastity cadre pitt shameful intellect skepticism surrender gordon banks soon edu lyme blood weight patients medical probably
Topic #4:
car 00 new sale price shipping offer 10 condition bike 50 sell asking good used 20 old buy 15 miles
Topic #5:
drive scsi ide disk controller drives hard floppy bus card mac hd motherboard cd isa cable pc boot apple mb
Topic #6:
thanks does know mail advance hi looking info anybody help appreciated address information email post need like appreciate wondering hello
Topic #7:
people don just 

In [6]:
H = nmf.components_
def build_wgraph(alpha=2):
    if alpha != 2:
        return [[int(cosine(H[i],H[j])[0][0] > alpha) for i in range(0, len(H))] for j in range(0, len(H))]
    else:
        return [[cosine(H[i],H[j])[0][0] for i in range(0, len(H))] for j in range(0, len(H))]


In [7]:
def thresh_vals(numbin):
    binz = []
    w = build_wgraph(2)
    chain = itertools.chain(*w)
    s =sorted(list(chain))
    val = n_topics*n_topics/numbin
    for i,v in enumerate(s):
        if i%val ==0: binz.append(v)
    return binz

In [8]:
def compare_graphs(graphA, graphB):
    return 47

In [9]:
seed = build_wgraph(0)

In [10]:
for t in thresh_vals(10):
    graphB = build_wgraph(t)
    compare_graphs(seed, graphB)
    seed = graphB

In [11]:
from scipy.sparse.csgraph import connected_components

In [12]:

size = 10
tv = thresh_vals(size)
ccA = connected_components(build_wgraph(tv[0]))
ccB = connected_components(build_wgraph(tv[1]))


In [111]:


def array_distance(A,B):
    count = 0
    for i,x in enumerate(A):
        if x == B[i]:
            count+=1
    return len(A)-count

def greedy_TV_build(to_consume,bins):
    if len(to_consume)>1:
        a = to_consume[0]
        b = to_consume[1]
        ccA = connected_components(build_wgraph(a))[1]
        ccB = connected_components(build_wgraph(b))[1]
        if not np.array_equal(ccA, ccB):
            distance = array_distance(ccA,ccB)
            if distance > 8:
                new_tv = [a + i*(b-a)/bins for i in range(0,bins)]
                return new_tv + greedy_TV_build( to_consume[1:], bins)
            else:
                return [a] + greedy_TV_build(to_consume[1:], bins)
        else:
            return greedy_TV_build(to_consume[1:], bins)
    elif len(to_consume) == 1:
        return to_consume
    else:
        return []
        

In [108]:
greedy_TV_build(thresh_vals(100),2)

[1,
 1,
 2,
 2,
 5,
 7,
 1,
 0.20189265946751342,
 0.22599850966429458,
 3,
 1.0000000000000009]

In [97]:
tv

[0.010853338553680987,
 0.023961721589683398,
 0.051877369962509752,
 0.06663866067784352,
 0.079680291988766916,
 0.10619111178753092,
 0.13813805951361693,
 0.16856385364420523,
 0.18771200531222493,
 0.99999999999999867]

In [112]:
tv = greedy_TV_build(thresh_vals(100),2)
for i in tv:
    ccB = connected_components(build_wgraph(i))[1]
    print ccB

[0 0 0 0 0 0 0 0 0 0]
[0 0 0 1 0 0 0 0 0 0]
[0 0 0 1 0 0 2 0 0 0]
[0 0 0 1 0 2 3 0 0 0]
[0 0 0 1 0 2 3 0 4 4]
[0 0 0 1 2 3 4 0 5 5]
[0 0 1 2 3 4 5 0 6 6]
[0 0 1 2 3 4 5 0 6 7]
[0 0 1 2 3 4 5 0 6 7]
[0 1 2 3 4 5 6 1 7 8]
[0 1 2 3 4 5 6 7 8 9]


In [77]:
print array_distance([0, 0, 0, 0 ,0 ,0 ,0 ,0 ,0 ,0],[0 ,0,0 ,0, 0, 0, 0 ,0 ,0, 0])

0


In [183]:
cc1 = connected_components(build_wgraph(tv[0]))[1]
cc2 = connected_components(build_wgraph(tv[1]))[1]


def populateTree(row_level, valid_community):
    if row_level > size-2 :
        return []
    else:
        children = []
        parent_community = connected_components(build_wgraph(tv[row_level]))[1]
        child_community = connected_components(build_wgraph(tv[row_level+1]))[1]
        unique_communities = list(set(parent_community)) 
        for unique_community in unique_communities:
            if valid_community == unique_community:
                indices = [i for i, x in enumerate(parent_community) if x == unique_community] #[8,9]
                seen_communities = []
                for i in indices: #8 and 9
                    if child_community[i] in seen_communities:
                        filter(lambda x: x['community'] == str(child_community[i]), children)[0]['indices'].append(i)
                    else:
                        community_to_find = child_community[i]
                        grow_my_children = populateTree(row_level+1, community_to_find)
                        if grow_my_children:
                            name = ""
                            children.append({"community":str(child_community[i]),"indices":[i],"name" : name , "children":grow_my_children, "hasChildren": True})
                        else:
                            name = " ".join([tfidf_feature_names[j] for j in nmf.components_[i].argsort()[:-n_top_words - 1:-1]])
                            children.append({"community":str(child_community[i]),"indices":[i],"size":500,"name":name, "hasChildren": False})
                        seen_communities.append(child_community[i])
        if len(children) == 1:
            try: 
                return children[0]['children']
            except:
                return children
        else:
            return children
    
flare = {"name" : "" , "children" : populateTree(0, 0)}
for child in flare['children']:
    recursiveNaming(child)
with open('demo.json', 'w') as outfile:
    json.dump(flare, outfile)

In [131]:
" ".join([tfidf_feature_names[j] for j in (nmf.components_[1]+nmf.components_[5]).argsort()[:-n_top_words - 1:-1]])

u'god drive scsi jesus bible ide hard disk controller drives believe faith christ christian christians floppy bus card does church'

In [181]:
def recursiveNaming(tree):
        i = tree['indices']
        base = nmf.components_[i[0]]
        if len(i) > 1:
            for ind in i[1:]:
                base = np.add(nmf.components_[ind], base)
        tree['name'] = " ".join([tfidf_feature_names[j] for j in base.argsort()[:-n_top_words - 1:-1]])
        if tree['hasChildren']:
            for child in tree['children']:
                    recursiveNaming(child)


('base:', array([  7.14221770e-01,   2.60603559e-01,   5.48973802e-03, ...,
         1.36373099e-05,   6.14298005e-03,   2.39430481e-03]))
('base:', array([  7.14221770e-01,   2.60603559e-01,   5.48973802e-03, ...,
         1.36373099e-05,   6.04919381e-03,   2.39430481e-03]))
('base:', array([  7.14221770e-01,   2.60603559e-01,   5.48973802e-03, ...,
         1.36373099e-05,   5.23927849e-03,   2.39430481e-03]))
('base:', array([  7.14221770e-01,   2.38551188e-01,   2.03644935e-03, ...,
         1.05264874e-05,   2.08233061e-03,   1.80104166e-03]))
('base:', array([  0.00000000e+00,   9.40665589e-02,   0.00000000e+00, ...,
         2.60561072e-06,   6.81639244e-05,   1.18861544e-03]))
('base:', array([  0.00000000e+00,   9.40665589e-02,   0.00000000e+00, ...,
         2.60561072e-06,   6.81639244e-05,   1.18861544e-03]))
('base:', array([  0.00000000e+00,   3.47678944e-02,   0.00000000e+00, ...,
         2.60561072e-06,   0.00000000e+00,   6.65479075e-06]))
('base:', array([  0.000000

In [182]:
flare

{'children': [{'children': [{'children': [{'children': [{'children': [{'children': [{'children': [{'community': '0',
               'hasChildren': False,
               'indices': [0],
               'name': u'game team games year players season play hockey win league teams baseball nhl player good detroit toronto runs best pitching',
               'size': 500},
              {'community': '1',
               'hasChildren': False,
               'indices': [1, 7],
               'name': u'geb dsl chastity n3jxp cadre pitt shameful intellect skepticism surrender gordon banks god soon edu people don jesus just think',
               'size': 500}],
             'community': '0',
             'hasChildren': True,
             'indices': [0, 1, 7],
             'name': u'geb pitt dsl chastity n3jxp cadre shameful intellect skepticism surrender gordon banks god soon edu game team don people think'},
            {'children': [{'community': '2',
               'hasChildren': False,
          

In [176]:
nmf.components_[1] 

array([ 0.        ,  0.01167469,  0.        , ...,  0.        ,
        0.        ,  0.00091785])

In [67]:
[a + i*(b-a)/bins for i in range(0,1)]

[1.0]